# Работа с изображением

## Цель

Попробовать применить SVD-разложение для сжатия изображений. Описание работы и результаты оформите в виде одного ipynb блокнота и выложите в свой гитхаб репозиторий

## Работа

Для начила подключим нужные библиотеки

In [1]:
from PIL import Image
import numpy as np

Теперь возьмём какое-нибудь изображение, например:

![](car_in_color.jpg)

Откроем его на Python при помощи библиотеки PIL (на самом деле pilow) и положим в переменную *img*

При исползовании метода show картинка откроется в стандартном редакторе

In [2]:
img = Image.open("car_in_color.jpg")
img.show()

Теперь переведём нашу *картинку* в массив чисел, используя print можем увидеть сам массив

In [3]:
arr = np.array(img)
# print(arr)

Для начала, разложим картинки на оттенки серго и попробуем применить SVD-разложение:

In [4]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

In [5]:
arr_gray = rgb2gray(arr)
# print(arr_gray)
# print(type(arr_gray))

In [6]:
arr_gray_img = Image.fromarray(arr_gray)
print(arr_gray_img)
arr_gray_img.show()

<PIL.Image.Image image mode=F size=1280x850 at 0x20C6AF435E0>


In [7]:
U, S, V = np.linalg.svd(arr_gray)
# print(U, S, V)

In [8]:
print(arr_gray.shape)
print(U.shape)
print(S.shape)
print(V.shape)

(850, 1280)
(850, 850)
(850,)
(1280, 1280)


In [9]:
reconstim = np.matrix(U[:, :500]) * np.diag(S[:500]) * np.matrix(V[:500, :])
reconstim = Image.fromarray(reconstim)
reconstim.show()

In [10]:
def create_pic(U, S, V, singul):
    reconstim = np.matrix(U[:, :singul]) * np.diag(S[:singul]) * np.matrix(V[:singul, :])
    reconstim = Image.fromarray(reconstim)
    reconstim.show()

In [11]:
for i in [1, 2, 10, 850]:
    create_pic(U, S, V, i)

In [12]:
# with open('car_in_gray.txt', 'w') as f:
#     with np.printoptions(threshold=np.inf):
#         f.write(str(U) + str(S) + str(V))
#     f.close()

Видим, что всё работает. 

Теперь применим подобное для основных rgb цветов и сложим картинку.

In [15]:
arr_r = arr[:,:,0]
arr_g = arr[:,:,1]
arr_b = arr[:,:,2]

In [44]:
U_r, S_r, V_r = np.linalg.svd(arr_r, full_matrices = False)
U_g, S_g, V_g = np.linalg.svd(arr_g, full_matrices = False)
U_b, S_b, V_b = np.linalg.svd(arr_b, full_matrices = False)

In [38]:
# Image.fromarray(np.matrix(U_r[:, :500]) * np.diag(S_r[:500]) * np.matrix(V_r[:500, :])).show()
# Image.fromarray(np.matrix(U_g[:, :500]) * np.diag(S_g[:500]) * np.matrix(V_g[:500, :])).show()
# Image.fromarray(np.matrix(U_b[:, :500]) * np.diag(S_b[:500]) * np.matrix(V_b[:500, :])).show()
Aaa = np.matrix(U_r[:, :500]) * np.diag(S_r[:500]) * np.matrix(V_r[:500, :])
print(np.shape(Aaa))
Aaa = Aaa.reshape(850, 1280, 1)
print(np.shape(Aaa))

(850, 1280)
(850, 1280)


In [56]:
# for i in [1, 850]:
#     r = np.matrix(U_r[:, :i]) * np.diag(S_r[:i]) * np.matrix(V_r[:i, :])
#     g = np.matrix(U_g[:, :i]) * np.diag(S_g[:i]) * np.matrix(V_g[:i, :])
#     b = np.matrix(U_b[:, :i]) * np.diag(S_b[:i]) * np.matrix(V_b[:i, :])
#     r = r.reshape(850, 1280, 1)
#     g = g.reshape(850, 1280, 1)
#     b = b.reshape(850, 1280, 1)
#     print(r.shape)
#     Ima = np.concatenate((np.concatenate((r, g), axis = 2), b), axis = 2)
#     Ima = Image.fromarray(Ima)
#     Image.fromarray(np.asarray(Ima, dtype=np.uint8)).show()


for r in range (1, 850, 170):
    Y_r_red = U_r[:,:r].dot(np.diag(S_r[:r])).dot(V_r[:r,:]) 
    Y_r_red[Y_r_red < 0] = 0
    Y_r_red[Y_r_red > 255] = 255
    Y_r_red = Y_r_red.reshape(850, 1280, 1)
    Y_r_green = U_g[:,:r].dot(np.diag(S_g[:r])).dot(V_g[:r,:]) 
    Y_r_green[Y_r_green < 0] = 0
    Y_r_green[Y_r_green > 255] = 255
    Y_r_green = Y_r_green.reshape(850, 1280, 1)
    Y_r_blue = U_b[:,:r].dot(np.diag(S_b[:r])).dot(V_b[:r,:]) 
    Y_r_blue[Y_r_blue < 0] = 0
    Y_r_blue[Y_r_blue > 255] = 255
    Y_r_blue = Y_r_blue.reshape(850, 1280, 1)
    Y_r = np.concatenate((np.concatenate((Y_r_red, Y_r_green), axis = 2), Y_r_blue), axis = 2)
    Image.fromarray(np.asarray(Y_r, dtype=np.uint8)).save(f'{r}.meow.jpg')
    

Программа работает